# Lab: Advanced Hyperparameter Tuning with Keras Tuner


Hyperparameter tuning is a critical step in machine learning model development. While manual tuning can work for simple models, automated hyperparameter optimization becomes essential as model complexity increases. This lab demonstrates advanced hyperparameter tuning techniques using Keras Tuner on the CIFAR-10 dataset, a more challenging image classification task compared to Fashion MNIST.

In this lab, you will:
- Work with the CIFAR-10 dataset (32x32 color images)
- Build a baseline convolutional neural network (CNN) model
- Use Keras Tuner's RandomSearch algorithm to find optimal hyperparameters
- Tune multiple hyperparameters including learning rate, dropout rate, activation functions, and layer configurations
- Compare baseline and tuned model performance

The RandomSearch tuner randomly samples from the hyperparameter space, making it a good choice when you want to explore a wide range of configurations without the computational overhead of more sophisticated algorithms.

Let's begin!


## Environment Setup

Before we begin, let's activate the ml-env conda environment to ensure we're using the correct package versions.


In [1]:
# Activate the ml-env conda environment
import sys
import os

# Note: In Jupyter notebooks, you may need to install ipykernel and register the environment
# conda activate ml-env
# conda install ipykernel
# python -m ipykernel install --user --name ml-env --display-name "Python (ml-env)"

print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")


Python version: 3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]
Python executable: c:\Users\hrrao\miniconda3\envs\ml-env\python.exe


## Download and prepare the dataset

Let us first load the [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) into your workspace. CIFAR-10 consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class. This is a more challenging dataset than Fashion MNIST as it contains color images and more complex visual patterns.


In [2]:
# Import required libraries
%load_ext tensorboard
from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Setup TensorBoard callback
tensorBoard_callback = keras.callbacks.TensorBoard("./tb_logs_cifar")


In [3]:
# Download the CIFAR-10 dataset and split into train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

print(f"Training images shape: {x_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test images shape: {x_test.shape}")
print(f"Test labels shape: {y_test.shape}")

# CIFAR-10 class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']
print(f"\nNumber of classes: {len(class_names)}")
print(f"Class names: {class_names}")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 15s 0us/step
Training images shape: (50000, 32, 32, 3)
Training labels shape: (50000, 1)
Test images shape: (10000, 32, 32, 3)
Test labels shape: (10000, 1)

Number of classes: 10
Class names: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


For preprocessing, you will normalize the pixel values to make the training converge faster. Since CIFAR-10 images are already in the range [0, 255], we'll normalize them to [0, 1].


In [4]:
# Normalize pixel values between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to integers (they come as 2D arrays from CIFAR-10)
y_train = y_train.flatten()
y_test = y_test.flatten()

print(f"Normalized training images range: [{x_train.min():.2f}, {x_train.max():.2f}]")
print(f"Training labels shape after flattening: {y_train.shape}")


Normalized training images range: [0.00, 1.00]
Training labels shape after flattening: (50000,)



## Baseline Performance

You will first establish a baseline performance using a CNN with arbitrarily selected hyperparameters. This will serve as a comparison point for the tuned model. The baseline model uses a simple CNN architecture suitable for CIFAR-10 classification.


In [5]:
# Build the baseline model using the Sequential API
b_model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), name='conv_1'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation='relu', name='conv_2'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation='relu', name='conv_3'),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu', name='dense_1'),  # Will tune this later
    keras.layers.Dropout(0.5),  # Will tune this later
    keras.layers.Dense(10, activation='softmax')
])

# Print model summary
b_model.summary()


c:\Users\hrrao\miniconda3\envs\ml-env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_1 (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2 (Conv2D)                 │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3 (Conv2D)                 │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 188,810 (737.54 KB)

 Trainable params: 188,810 (737.54 KB)

 Non-trainable params: 0 (0.00 B)

As shown, we hardcoded several hyperparameters including the number of units in the dense layer, dropout rate, and learning rate. You will see how to automatically tune these and more in the upcoming sections.


Let's setup the loss, metrics, and optimizer. The learning rate is set to `0.001` for the baseline, but this will be tuned automatically later.


In [6]:
# Setup the training parameters
b_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])


Now you can start training the baseline model. We've set the number of epochs to 10 for demonstration purposes.


In [7]:
# Number of training epochs
NUM_EPOCHS = 10

# Train the baseline model
b_history = b_model.fit(x_train, y_train, 
                        epochs=NUM_EPOCHS, 
                        validation_split=0.2, 
                        callbacks=[tensorBoard_callback],
                        verbose=1)


Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 24s 17ms/step - accuracy: 0.2809 - loss: 1.9227 - val_accuracy: 0.5170 - val_loss: 1.3520
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 28s 22ms/step - accuracy: 0.4984 - loss: 1.3887 - val_accuracy: 0.5709 - val_loss: 1.2021
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 35s 18ms/step - accuracy: 0.5679 - loss: 1.2175 - val_accuracy: 0.6181 - val_loss: 1.0772
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.6068 - loss: 1.1079 - val_accuracy: 0.6422 - val_loss: 0.9998
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 16ms/step - accuracy: 0.6350 - loss: 1.0371 - val_accuracy: 0.6475 - val_loss: 0.9793
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.6537 - loss: 0.9806 - val_accuracy: 0.6442 - val_loss: 1.0448
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.6742 - loss: 0.9299 - val_accuracy: 0.6756 - val_loss: 0.9385
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.6913 -

Finally, evaluate the baseline model on the test set to see its performance.


In [8]:
# Evaluate baseline model on the test set
b_eval_dict = b_model.evaluate(x_test, y_test, return_dict=True, verbose=1)


313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7018 - loss: 0.8765


Let's define a helper function for displaying the results so it's easier to compare later.


In [9]:
# Define helper function
def print_results(model, model_name, layer_name, eval_dict):
    '''
    Prints the values of the hyperparameters to tune, and the results of model evaluation

    Args:
        model (Model) - Keras model to evaluate
        model_name (string) - arbitrary string to be used in identifying the model
        layer_name (string) - name of the layer to tune
        eval_dict (dict) - results of model.evaluate
    '''
    print(f'\n{model_name}:')
    
    # Get dense layer units
    dense_layer = model.get_layer(layer_name)
    print(f'Number of units in {layer_name}: {dense_layer.units}')
    
    # Get dropout rate
    dropout_layer = None
    for layer in model.layers:
        if isinstance(layer, keras.layers.Dropout):
            dropout_layer = layer
            break
    if dropout_layer:
        print(f'Dropout rate: {dropout_layer.rate}')
    
    # Get learning rate
    print(f'Learning rate for the optimizer: {model.optimizer.learning_rate.numpy()}')
    
    # Print evaluation metrics
    for key, value in eval_dict.items():
        print(f'{key}: {value:.4f}')

# Print results for baseline model
print_results(b_model, 'BASELINE MODEL', 'dense_1', b_eval_dict)



BASELINE MODEL:
Number of units in dense_1: 128
Dropout rate: 0.5
Learning rate for the optimizer: 0.0010000000474974513
accuracy: 0.6963
loss: 0.8816


That's it for getting the results for a single set of hyperparameters. As you can see, manually trying different combinations of hyperparameters (learning rate, dropout rate, number of units, activation functions, etc.) would be very time-consuming. Keras Tuner automates this process by searching through the hyperparameter space efficiently. You will see how this is done in the next sections.


## Keras Tuner with RandomSearch

To perform hypertuning with Keras Tuner, you will need to:

* Define the model builder function
* Select which hyperparameters to tune
* Define the search space for each hyperparameter
* Choose a search strategy (RandomSearch in this case)


### Install and import packages

You will start by installing and importing the required packages.


In [11]:
# Install Keras Tuner (uncomment if needed)
# !pip install -q -U keras-tuner

# Import required packages
import tensorflow as tf
import keras_tuner as kt


### Define the model builder function

The model builder function defines the hyperparameter search space in addition to the model architecture. This function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In this lab, you will tune several hyperparameters:
* The number of units in the Dense layer (using `Int()` method)
* The dropout rate (using `Float()` method)
* The learning rate (using `Choice()` method)
* The activation function for the dense layer (using `Choice()` method)

The RandomSearch tuner will randomly sample combinations of these hyperparameters to find the optimal configuration.


In [12]:
def model_builder(hp):
    '''
    Builds the model and sets up the hyperparameters to tune.

    Args:
        hp - Keras tuner object

    Returns:
        model with hyperparameters to tune
    '''
    # Initialize the Sequential API and start stacking the layers
    model = keras.Sequential()
    
    # Convolutional layers (keeping architecture similar to baseline)
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), name='conv_1'))
    model.add(keras.layers.MaxPooling2D(2, 2))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', name='conv_2'))
    model.add(keras.layers.MaxPooling2D(2, 2))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', name='conv_3'))
    model.add(keras.layers.Flatten())
    
    # Tune the number of units in the Dense layer
    # Choose an optimal value between 64-256
    hp_units = hp.Int('units', min_value=64, max_value=256, step=32)
    
    # Tune the activation function for the dense layer
    hp_activation = hp.Choice('activation', values=['relu', 'tanh', 'elu'])
    
    model.add(keras.layers.Dense(units=hp_units, activation=hp_activation, name='tuned_dense_1'))
    
    # Tune the dropout rate
    # Choose an optimal value between 0.2 and 0.6
    hp_dropout = hp.Float('dropout', min_value=0.2, max_value=0.6, step=0.1)
    model.add(keras.layers.Dropout(hp_dropout))
    
    # Output layer
    model.add(keras.layers.Dense(10, activation='softmax'))
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, 0.0001, or 0.00001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
    
    return model


In [13]:
# Instantiate the RandomSearch tuner
tuner = kt.RandomSearch(model_builder,
                        objective='val_accuracy',
                        max_trials=10,  # Number of hyperparameter combinations to try
                        executions_per_trial=1,  # Number of models to train per trial
                        directory='kt_dir_cifar',
                        project_name='kt_random_search')


Let's see a summary of the hyperparameters that you will tune:


In [14]:
# Display hypertuning settings
tuner.search_space_summary()


Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'elu'], 'ordered': False}
dropout (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.6, 'step': 0.1, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001, 1e-05], 'ordered': True}


You can pass in a callback to stop training early when a metric is not improving. Below, we define an [EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) callback to monitor the validation loss and stop training if it's not improving after 5 epochs.


In [15]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


Setup TensorBoard callback for monitoring the hyperparameter search process.


In [16]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir='./keras_tuner_cifar', update_freq='batch')


You will now run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callbacks above. This will take some time to run as it tries multiple hyperparameter combinations.


In [17]:
# Perform hypertuning
tuner.search(x_train, y_train, 
             epochs=NUM_EPOCHS, 
             validation_split=0.2, 
             callbacks=[stop_early, tensorboard_callback],
             verbose=1)


Trial 10 Complete [00h 06m 07s]
val_accuracy: 0.6991000175476074

Best val_accuracy So Far: 0.7121000289916992
Total elapsed time: 01h 26m 04s


You can get the top performing model with the [get_best_hyperparameters()](https://keras-team.github.io/keras-tuner/documentation/tuners/#get_best_hyperparameters-method) method.


In [18]:
# Get the optimal hyperparameters from the results
best_hps = tuner.get_best_hyperparameters()[0]

print(f"""
The hyperparameter search is complete. The optimal hyperparameters are:
- Number of units in the densely-connected layer: {best_hps.get('units')}
- Activation function: {best_hps.get('activation')}
- Dropout rate: {best_hps.get('dropout')}
- Learning rate for the optimizer: {best_hps.get('learning_rate')}
""")



The hyperparameter search is complete. The optimal hyperparameters are:
- Number of units in the densely-connected layer: 160
- Activation function: tanh
- Dropout rate: 0.5
- Learning rate for the optimizer: 0.001



## Build and train the tuned model

Now that you have the best set of hyperparameters, you can rebuild the hypermodel with these values and retrain it.


In [19]:
# Build the model with the optimal hyperparameters
h_model = tuner.hypermodel.build(best_hps)
h_model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_1 (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2 (Conv2D)                 │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3 (Conv2D)                 │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tuned_dense_1 (Dense)           │ (None, 160)            │       164,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 160)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,610 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 221,930 (866.91 KB)

 Trainable params: 221,930 (866.91 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Train the hypertuned model
h_history = h_model.fit(x_train, y_train, 
                        epochs=NUM_EPOCHS, 
                        validation_split=0.2,
                        verbose=1)


Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 29s 22ms/step - accuracy: 0.3320 - loss: 1.8023 - val_accuracy: 0.5552 - val_loss: 1.2539
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 24s 20ms/step - accuracy: 0.5674 - loss: 1.2236 - val_accuracy: 0.6005 - val_loss: 1.1154
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 25s 20ms/step - accuracy: 0.6292 - loss: 1.0530 - val_accuracy: 0.6631 - val_loss: 0.9598
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.6706 - loss: 0.9392 - val_accuracy: 0.6724 - val_loss: 0.9448
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.7006 - loss: 0.8564 - val_accuracy: 0.6803 - val_loss: 0.9266
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 25s 20ms/step - accuracy: 0.7218 - loss: 0.7955 - val_accuracy: 0.7038 - val_loss: 0.8568
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 26s 20ms/step - accuracy: 0.7385 - loss: 0.7449 - val_accuracy: 0.7103 - val_loss: 0.8457
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 26s 21ms/step - accuracy: 0.7578 -

You will then get its performance against the test set.


In [21]:
# Evaluate the hypertuned model against the test set
h_eval_dict = h_model.evaluate(x_test, y_test, return_dict=True, verbose=1)


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7030 - loss: 0.9077


Now we can compare the results we got with the baseline model. The tuned model should show improved performance or similar performance with better hyperparameter choices. Results may vary, but you will typically see improvements in validation and test accuracy.


In [22]:
# Print results of the baseline and hypertuned model
print_results(b_model, 'BASELINE MODEL', 'dense_1', b_eval_dict)
print_results(h_model, 'HYPERTUNED MODEL', 'tuned_dense_1', h_eval_dict)



BASELINE MODEL:
Number of units in dense_1: 128
Dropout rate: 0.5
Learning rate for the optimizer: 0.0010000000474974513
accuracy: 0.6963
loss: 0.8816

HYPERTUNED MODEL:
Number of units in tuned_dense_1: 160
Dropout rate: 0.5
Learning rate for the optimizer: 0.0010000000474974513
accuracy: 0.6944
loss: 0.9312


## Summary

In this lab, you learned how to:
- Use Keras Tuner's RandomSearch algorithm for hyperparameter optimization
- Tune multiple hyperparameters simultaneously (units, dropout, learning rate, activation function)
- Compare baseline and tuned model performance
- Work with the CIFAR-10 dataset for image classification

The RandomSearch approach provides a straightforward way to explore the hyperparameter space. For more complex models or when computational resources are limited, you might want to consider other tuners like Hyperband or Bayesian Optimization, which can be more efficient in finding optimal hyperparameters.


In [23]:
# Optional: View TensorBoard logs
# %tensorboard --logdir ./tb_logs_cifar
